In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

%matplotlib inline

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Classroom/mnist_sequential_model

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Classroom/spam.csv', delimiter = ',', encoding = 'latin-1')
df.head()

/content/drive/MyDrive/Colab Notebooks/Classroom/mnist_sequential_model


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
print(Y)
print(Y.shape)
Y = Y.reshape(-1,1)

[0 0 1 ... 0 0 0]
(5572,)


In [ ]:
print(Y.shape)

(5572, 1)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequence_matrix = sequence.pad_sequences(sequences, maxlen= max_len)

In [ ]:
def RNN() :

  inputs = Input(name='inputs', shape = [max_len])
  layer = Embedding(max_words,50,input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256, name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1, name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs = inputs, outputs = layer)
  return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           50000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257   

In [ ]:
model.fit(sequence_matrix, Y_train, batch_size = 128, epochs = 10, validation_split = 0.2, callbacks = [EarlyStopping(monitor = 'val_loss')])

Epoch 1/10
30/30 [==============================] - 9s 205ms/step - loss: 0.4479 - accuracy: 0.8108 - val_loss: 0.1418 - val_accuracy: 0.9557
Epoch 2/10
30/30 [==============================] - 6s 186ms/step - loss: 0.0989 - accuracy: 0.9781 - val_loss: 0.0536 - val_accuracy: 0.9842
Epoch 3/10
30/30 [==============================] - 6s 185ms/step - loss: 0.0418 - accuracy: 0.9876 - val_loss: 0.0538 - val_accuracy: 0.9831


In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen = max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix, Y_test)

27/27 [==============================] - 1s 19ms/step - loss: 0.0588 - accuracy: 0.9821


In [ ]:
print('Test set\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
 Loss: 0.059
 Accuracy: 0.982
